In [ ]:
from selenium import webdriver
import requests
import re

In [ ]:
browser = webdriver.Chrome()
URL = 'https://mei.org.uk/step-aea-solutions'
browser.get(URL)

In [ ]:
browser.find_element_by_xpath('//button[text()="STEP Solutions"]').click()
browser.switch_to_window(browser.window_handles[1])
browser.current_url

In [ ]:
browser.find_element_by_link_text('STEP past paper worked solutions').click()
browser.current_url

In [ ]:
elements = browser.find_elements_by_partial_link_text('STEP solutions')
groups = []
for element in elements:
    groups.append(element.get_attribute("href"))

groups

In [ ]:
pattern = r"STEP (1|I)+: \d\d\d\d"
all_urls = []

login_url = "https://2017.integralmaths.org/login/index.php"
payload = {
    'username': 'mei-step',
    'password': '' # edit this with correct password when running notebook
}

for url_group in groups:
    browser.get(url_group)
    
    papers = []
    for paper in browser.find_elements_by_partial_link_text('STEP '):
        if re.match(pattern,paper.get_property('textContent')):
            papers.append([paper.get_attribute("href"), paper.get_property('textContent')])
    
    for url_paper, paper_name in papers:
        browser.get(url_paper)
        questions = browser.find_elements_by_partial_link_text('Question ')
        for question in questions:
            question.click()
            browser.switch_to_window(browser.window_handles[2])

            url = browser.current_url
            filename = paper_name+'-'+url.split('/')[-1]
            all_urls.append([url, filename])
            with requests.Session() as s:
                s.post(login_url, data=payload)
                r = s.get(url, allow_redirects=True, stream=True)
                open(filename, 'wb').write(r.content)

            browser.close()
            browser.switch_to_window(browser.window_handles[1])

In [ ]:
len(all_urls)

In [ ]:
with open('answer_urls.txt','w') as f:
    f.writelines([name+';;'+url+'\n' for url,name in all_urls])